# Example of Model Post-Processing

**This is an example of the low-level TUFLOW-FV python tools, which provides all the base methods to accessing TUFLOW-FV Results.

The easier to use interface is the "Xarray" accessor - see "tfv_tools_with_xarray.ipynb"**

This is a notebook that provides a number of examples on how the tfv package can be used to view and post-process tuflow fv model results.

First we start by importing the usual suspects: `numpy` and `matplotlib`. <br>We'll also make use of the `pathlib` module to assist with managing file-paths, although this is entirely optional! 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

From the tfv package, lets start with the `FvExtractor` object. <br> 
This is the arguably the most important module in the tfv package. It is used for extracting data from a tuflow fv spatial model output, including:
- Extracting sheet cell or nodal values (i.e., 2D cell center or nodal values)
- Extract profile values (i.e., 1D vertical profiles)
- Extract curtain grid or curtain cells (i.e., 2D slices along a chainage)

In [ ]:
from tfv.extractor import FvExtractor

### Load a model result
This example uses our TUFLOW FV Tamar Island model.<br> Note that we typically use `Path` objects to manage unix/dos path format and other benefits, but this is optional and you could use text strings instead.

In [ ]:
# model_folder = Path(r'./data')
model_folder = Path(r"C:\Users\alexander.waterhouse\Python\tfv\working")
model_file = 'HYD_002_extended.nc'

Now we'll initialise our extractor object that will be used throughout the rest of this notebook

In [ ]:
xtr = FvExtractor(model_folder / model_file)

The ```FvExtractor``` object uses `xarray` under the hood in the latest versions of the tfv package (which this notebook tutorial is based on). <br>
Xarray provides are great number of built in ways to subset and extract data - and you can utilise the raw xarray object itself. <br> You can view the underlying dataset anytime:

In [ ]:
xtr.ds

A common task is to cut down a large tuflow-fv result into a much smaller subset, for example to view the file in the QGIS TUFLOW Viewer, or to transfer over a network. <br> The following cell shows how we can leverage Xarray's inbuilt functionality to subset a 6-hr period from the result, and then export a much smaller netcdf result.  

In [ ]:
start = '2011-02-03 12:00'
stop = '2011-02-03 18:00'
period = slice(start, stop) # We use a slice object with the start and stop times to specify the period we want

ds_sliced = xtr.ds.sel(Time=period)  # Now we have a "cut" down xarray dataset

ds_sliced

# To export this to a new netcdf file, use:
# ds_sliced.to_netcdf('./example_sliced_tuflowfv_result.nc')

### Extracting data
Now that we have our `xtr` object loaded, let's look at some common methods to pull out data. <br>
These can be used in a variety of ways, so please experiment! 

In [ ]:
# Pull out cell center depth-averaged data for the 5th timestep
cell_center_result = xtr.get_sheet_cell('SAL', 5)

# Pull out node depth-averaged data for a specific date
node_result = xtr.get_sheet_node('SAL', '2011-02-05 12:00:00')

# Pull out a profile
profile_result = xtr.get_profile_cell('TEMP', '2011-02-02 03:00', (159.09, -31.39))

The examples above are building blocks to helping you understand your model result. However, an array of cell center values isn't very useful, unless you're aiming to do further processing to the values. <br>So, let's move onto looking at how we can plot our data!

# Viewing the data

The tfv.visual module has several convenient  functions to plot model results. 
<br>It is easiest to import them all with '*' <br>Please note that this is generally not good practice in python, and should be limited to small packages (like `tfv`!)

In [ ]:
from tfv.visual import *

### Spatial Plots

Let's start by looking at velocity (current) in the model. <br>There are some typical plotting settings in the cell below, that are used to adjust colormaps and vector sizes - this is a good one to copy and save for later use!

In [ ]:
col_spec = dict(cmap=cm.turbo, clim=[0.0, 0.25]) # Colormap and color scale
vec_spec = dict(units='dots', scale_units='dots', scale=1/100, width=0.5,
                headwidth=10, headlength=10, angles='uv', pivot='middle')  # Vector settings

def style_ax(ax):
    ax.set_xlabel('Longitude (°)')
    ax.set_ylabel('Latitude (°)')
    
    ax.set_aspect('equal')
    for tick in ax.get_xticklabels():
            tick.set_rotation(0)

Now we can draw our plot. The general steps are:
1. Create a matplotlib figure and axis. We simply use `fig, ax = plt.subplots` for this. 
2. Call our chosen plotting function. We'll use `SheetPatch`. <br>Note, it's important for the next step that we assign this to a variable - e.g., `sheet = SheetPatch(...)`
3. Finally, call `(obj).set_time_current` to modifiy the time being plotted

We will plot velocity. While TUFLOW FV outputs `V_x` and `V_y` separately, the `tfv` tools will recognise `V` as the combination of these two variables (Current Speed). For the vectors however, we need to tell it to use the components, so we pass a list of `['V_x', 'V_y']`

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
sheet = SheetPatch(ax, xtr, 'V', shading='interp', **col_spec) # See note below
vector = SheetVector(ax, xtr, ['V_x', 'V_y'], **vec_spec)

sheet.set_time_current('2011-02-03 12:00') # Pick a date to look at the result
vector.set_time_current('2011-02-03 12:00') # Pick a date to look at the result
style_ax(ax)

<i> Note on `SheetPatch`: the optional shading argument can be equal to `interp` or `flat` for a smooth or cell based figure. <br>
If plotting with "shading=flat", you may wish to hide the mesh outline using `edgecolor=face` as well!
<br>

### Vertical Averaging

What if we were interested in how the model result varies vertically? <br>We can provide the optional arguments to perform depth-averaging logic: `datum` and `limits` <br>
A good description of these are provided here: https://fvwiki.tuflow.com/index.php?title=Depth_Averaging_Results

As an example, we'll plot a figure with two subplots showing water-temperature for the top 2m (surface) and bottom 2m (near seabed)

In [ ]:
temp_color_spec = {'clim':(10, 20), 'cmap': 'inferno'}

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8,4), constrained_layout=True)
fig.suptitle('Modelled Temperature', fontweight='bold', y=1.06) # y is used to adjust the label position.

# Surface Temperature - top 2m
ax = axes[0]; style_ax(ax)
sheet1 = SheetPatch(ax, xtr, 'TEMP', datum='depth', limits=(0, 2), shading='interp', **temp_color_spec)
ax.set_title('Surface')

# Near Seabed Temperature - bottom 2m 
ax = axes[1]; style_ax(ax)
sheet2 = SheetPatch(ax, xtr, 'TEMP', datum='height', limits=(0, 2), shading='interp', **temp_color_spec)
ax.set_title('Seabed')

# Now let's update the time
time = '2011-02-03 12:00'
sheet1.set_time_current(time)
sheet2.set_time_current(time)

# Add a colorbar 
# (Note, you must supply something to matplotlib's colorbar function so it knows what the range is).
# For SheetPatch, this is sheet.patch; for SheetContour, this is sheet.cont
cbar = fig.colorbar(sheet1.patch, ax=axes.ravel().tolist())
cbar.set_label("Temperature (°C)")

# Plotting Custom Data

What if we want to do some statistics on the model results and then plot them. For example, the mean surface temperature during the simulation?

No issue - first we need to extract the surface temperature and then pass the data through to the plotting functions. <br> <b> We're going to extract both the cell-centered AND node values - we'll explain why in the next cell.

In [ ]:
data_cells = np.zeros((xtr.nt, xtr.nc2)) # ntimesteps x ncells
data_nodes = np.zeros((xtr.nt, xtr.nv2)) # ntimesteps x nnodes (aka num verticies, or nv)

for t in range(xtr.nt):
    data_cells[t, :] = xtr.get_sheet_cell('TEMP', t, datum='depth', limits=(0,1))
    data_nodes[t, :] = xtr.get_sheet_node('TEMP', t, datum='depth', limits=(0,1))

mean_temp_cells = data_cells.mean(axis=0) # Now we can average
mean_temp_nodes = data_nodes.mean(axis=0) # Now we can average

Now we can plot our data. We can choose to use `SheetPatch` or `SheetContour`. <br>
<b>Note</b>:
- `SheetPatch` with `shading=flat` uses cell-centered results to draw the figure. So the data must be cell centered
- `SheetContour` OR `SheetPatch` with `shading=interp` uses node results, so we need to supply nodal results. 

To pass custom data to the plot objects, we just replace the variable input with the data itself

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(10,8), constrained_layout=True)
# fig.suptitle('Modelled Temperature', fontweight='bold', y=1.06) # y is used to adjust the label position.

# Cells with SheetPatch
ax = axes[0]; style_ax(ax)
sheet1 = SheetPatch(ax, xtr, mean_temp_cells, shading='flat', **temp_color_spec)
ax.set_title('Cells with SheetPatch')

# Nodes with SheetPatch
ax = axes[1]; style_ax(ax)
sheet2 = SheetPatch(ax, xtr, mean_temp_nodes, shading='interp', **temp_color_spec)
ax.set_title('Nodes with SheetPatch')

# Node Based
ax = axes[2]; style_ax(ax)
sheet3 = SheetContour(ax, xtr, mean_temp_nodes, levels=np.arange(10, 20, 0.25), **temp_color_spec)
ax.set_title('Nodes with SheetContour')

plt.show()

# Learning More

This notebook has been an example on some of the high-level uses of the `tfv` tools. 
Plenty of other examples are provided in the user group! 